In [1]:
import string
import pandas as pd

In [2]:
# Carrega dataframe
# Cada linha é uma proposicao
# A coluna id é o código único no sistema interno da Câmara dos Deputados
# Popularmente as proposicoes sao conhecidas pela juncao de: siglaTipo + numero + / + ano (ex. PL 2342/2010)
# Entao vou criar um campo artificial para referencia, com mais o id -> codigo_unico

In [3]:
proposicoes = pd.read_csv("resultados/projetos_camara_2010_2018.csv",\
                          sep=',',\
                          encoding = 'utf-8', \
                          converters={'ano': lambda x: str(x),
                                      'id': lambda x: str(x), 
                                      'idTipo': lambda x: str(x),
                                      'numero': lambda x: str(x)
                                      })

In [4]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 7 columns):
ano          146322 non-null object
ementa       144347 non-null object
id           146322 non-null object
idTipo       146322 non-null object
numero       146322 non-null object
siglaTipo    146321 non-null object
uri          146320 non-null object
dtypes: object(7)
memory usage: 7.8+ MB


In [5]:
def cria(linha):
    tipo = str(linha["siglaTipo"])
    numero = str(linha['numero'])
    ano = str(linha['ano'])
    id_ = str(linha['id'])
    documento = tipo + ' ' + numero + '/' + ano + ' - ' + id_
    return documento

proposicoes["codigo_unico"] = proposicoes.apply(cria, axis=1)

In [6]:
# Trabalha com uma copia da coluna ementa

In [7]:
proposicoes["ementa_copia"] = proposicoes["ementa"]

In [8]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 9 columns):
ano             146322 non-null object
ementa          144347 non-null object
id              146322 non-null object
idTipo          146322 non-null object
numero          146322 non-null object
siglaTipo       146321 non-null object
uri             146320 non-null object
codigo_unico    146322 non-null object
ementa_copia    144347 non-null object
dtypes: object(9)
memory usage: 10.0+ MB


In [9]:
# Rotinas para tirar pontuacao e deixar minuscula

In [10]:
def limpeza1(linha):
    documento = str(linha["ementa_copia"])
    documento = documento.strip()
    documento = documento.replace(',', '')
    documento = documento.replace('.', '')
    documento = documento.replace('-', '')
    documento = documento.replace('/', '')
    documento = documento.replace(';', '')
    documento = documento.replace('"', '')
    documento = documento.translate(string.punctuation)
    documento = documento.lower()
    return documento
    
proposicoes["ementa_copia"] = proposicoes.apply(limpeza1, axis=1)

In [11]:
# Cria arquivo para checagem visual

In [73]:
proposicoes.to_excel('resultados/teste.xlsx',sheet_name='Sheet1')

In [12]:
proposicoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 9 columns):
ano             146322 non-null object
ementa          144347 non-null object
id              146322 non-null object
idTipo          146322 non-null object
numero          146322 non-null object
siglaTipo       146321 non-null object
uri             146320 non-null object
codigo_unico    146322 non-null object
ementa_copia    146322 non-null object
dtypes: object(9)
memory usage: 10.0+ MB


In [13]:
#Words_array

In [14]:
def limpeza2(linha):
    documento = str(linha["ementa_copia"])
    documento = documento.split(' ')
    return documento
      
proposicoes["words_array"] = proposicoes.apply(limpeza2, axis=1)

In [15]:
proposicoes.reset_index().head()

,index,ano,ementa,id,idTipo,numero,siglaTipo,uri,codigo_unico,ementa_copia,words_array
0,0,2010,Comunica o Excelentíssimo Senhor Presidente da...,465465,412,36,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 36/2010 - 465465,comunica o excelentíssimo senhor presidente da...,"[comunica, o, excelentíssimo, senhor, presiden..."
1,1,2010,"Abre crédito extraordinário, em favor da Presi...",465466,291,480,MPV,https://dadosabertos.camara.leg.br/api/v2/prop...,MPV 480/2010 - 465466,abre crédito extraordinário em favor da presid...,"[abre, crédito, extraordinário, em, favor, da,..."
2,2,2010,Comunica o Excelentíssimo Senhor Presidente da...,465467,415,38,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 38/2010 - 465467,comunica o excelentíssimo senhor presidente da...,"[comunica, o, excelentíssimo, senhor, presiden..."
3,3,2010,Submete à deliberação do Congresso Nacional Pr...,465469,134,42,MSC,https://dadosabertos.camara.leg.br/api/v2/prop...,MSC 42/2010 - 465469,submete à deliberação do congresso nacional pr...,"[submete, à, deliberação, do, congresso, nacio..."
4,4,2010,Concede auxílio especial e bolsa especial aos ...,465470,139,6720,PL,https://dadosabertos.camara.leg.br/api/v2/prop...,PL 6720/2010 - 465470,concede auxílio especial e bolsa especial aos ...,"[concede, auxílio, especial, e, bolsa, especia..."


In [16]:
# Cria dataframe apenas com o codigo_unico e words_array

In [17]:
prop_selec = proposicoes[['codigo_unico', 'words_array']]

In [18]:
prop_selec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 2 columns):
codigo_unico    146322 non-null object
words_array     146322 non-null object
dtypes: object(2)
memory usage: 2.2+ MB


In [19]:
# Remove linhas em branco

In [26]:
prop_selec_cleaned = prop_selec[prop_selec['words_array'] != ""]
prop_selec_cleaned.reset_index().head()

,index,codigo_unico,words_array
0,0,MSC 36/2010 - 465465,"[comunica, o, excelentíssimo, senhor, presiden..."
1,1,MPV 480/2010 - 465466,"[abre, crédito, extraordinário, em, favor, da,..."
2,2,MSC 38/2010 - 465467,"[comunica, o, excelentíssimo, senhor, presiden..."
3,3,MSC 42/2010 - 465469,"[submete, à, deliberação, do, congresso, nacio..."
4,4,PL 6720/2010 - 465470,"[concede, auxílio, especial, e, bolsa, especia..."


In [27]:
prop_selec_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146322 entries, 0 to 146321
Data columns (total 2 columns):
codigo_unico    146322 non-null object
words_array     146322 non-null object
dtypes: object(2)
memory usage: 3.3+ MB


In [34]:
# Transposicao

In [35]:
df_transp = prop_selec_cleaned.set_index('codigo_unico')['words_array'].to_frame().T

In [36]:
df_transp.reset_index().head()

codigo_unico,index,MSC 36/2010 - 465465,MPV 480/2010 - 465466,MSC 38/2010 - 465467,MSC 42/2010 - 465469,PL 6720/2010 - 465470,PL 6721/2010 - 465481,MSC 43/2010 - 465485,PL 6722/2010 - 465486,REQ 68/2010 - 465498,...,TVR 479/2018 - 2190318,TVR 480/2018 - 2190319,TVR 481/2018 - 2190320,TVR 482/2018 - 2190321,MSC 757/2018 - 2190322,MSC 795/2018 - 2190323,MSC 796/2018 - 2190324,MSC 799/2018 - 2190326,MSC 798/2018 - 2190327,MSC 797/2018 - 2190344
0,words_array,"[comunica, o, excelentíssimo, senhor, presiden...","[abre, crédito, extraordinário, em, favor, da,...","[comunica, o, excelentíssimo, senhor, presiden...","[submete, à, deliberação, do, congresso, nacio...","[concede, auxílio, especial, e, bolsa, especia...","[transforma, cargos, de, promotor, de, justiça...","[submete, à, deliberação, do, congresso, nacio...","[institui, o, programa, nacional, de, fomento,...","[requer, seja, enviada, diligência, ao, municí...",...,"[submete, à, apreciação, do, congresso, nacion...","[submete, à, apreciação, do, congresso, nacion...","[submete, à, apreciação, do, congresso, nacion...","[submete, à, apreciação, do, congresso, nacion...","[acordo, de, cooperação, técnica, entre, o, go...","[do, ajuste, complementar, ao, acordo, entre, ...","[do, acordo, entre, a, república, federativa, ...","[submete, à, deliberação, do, congresso, nacio...","[submete, à, , apreciação, dos, senhores, memb...","[texto, do, ajuste, complementar, ao, acordo, ..."


In [37]:
df_transp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, words_array to words_array
Columns: 146322 entries, MSC 36/2010 - 465465 to MSC 797/2018 - 2190344
dtypes: object(146322)
memory usage: 1.1+ MB
